In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from tqdm.notebook import tqdm

path = "./chromedriver.exe"
url = "https://www.yogiyo.co.kr/mobile/#/"

driver = wb.Chrome(path)
driver.get(url)

btn = driver.find_element_by_xpath("/html/body/div[2]/div[1]/div/div[2]/div/form/input")
btn.click()
time.sleep(0.5)

btn = driver.find_element_by_css_selector("#button_search_address > button.btn-search-location-cancel.btn-search-location.btn.btn-default")
btn.click()
time.sleep(0.5)

input_search = driver.find_element_by_xpath("/html/body/div[2]/div[1]/div/div[2]/div/form/input")
input_search.send_keys("서울특별시 강남구 논현동 280 이전노점28")
# 서울특별시 관악구 미성동 608-35
time.sleep(0.5)

btn = driver.find_element_by_xpath('//*[@id="button_search_address"]/button[2]')
btn.click()

In [2]:
#전체보기 2
#1인분 주문 3
#프랜차이즈 4
#치킨 5
#피자/양식 6
#중국집 7
#한식 8
#일식/돈까스 9
#돈까스족발보쌈 10
#야식분식카페 11
#디저트편의점 12
#마트 13

btn = driver.find_element_by_css_selector("#category > ul > li:nth-child(8) > span")
btn.click()

time.sleep(1)

#리뷰 많은 순 보기
btn = driver.find_element_by_css_selector("#content > div > div.row.restaurant-list-info > div.list-option > div > select > option:nth-child(3)")
btn.click()
time.sleep(0.5)

In [ ]:
first_restaurant_number = 59
last_restaurant_number = 61
total_restaurant = last_restaurant_number - first_restaurant_number

for restaurant_num in tqdm(range(first_restaurant_number,last_restaurant_number,1)):
     
    print(str(first_restaurant_number) + "번 식당부터, "+ str(last_restaurant_number) + "번 식당까지, "
          + str(total_restaurant) + "개의 식당 중, "
          + str(restaurant_num) + "번 식당 크롤링 중")
    
    #식당 리스트
    restaurant_list = driver.find_element_by_css_selector(f"#content > div > div:nth-child(5) > div > div > div:nth-child({restaurant_num}) > div")
    restaurant_list.click()
    
    time.sleep(1)
    
    #식당 이름 가져오기
    restaurant_name =  driver.find_element_by_css_selector("#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > div.restaurant-info > div.restaurant-title > span").text.strip()

    #식당 리뷰 페이지 넘어가기
    restaurant_reviewpage = driver.find_element_by_css_selector("#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > ul > li:nth-child(2)")
    restaurant_reviewpage.click()
    time.sleep(0.5)
    
    # 고객 댓글 수 체크
    customer_num = driver.find_element_by_css_selector("#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > div:nth-child(6) > div.review-count > div > strong:nth-child(1)").text.strip()
    customer_num = int(customer_num)
    
    print(restaurant_name + " " + str(customer_num) +"개의 리뷰 크롤링")
    
    if(customer_num>300):
        repeat_num_per_restaurant = int(int(customer_num)/300)+2
    else:
        repeat_num_per_restaurant = 2
    
    taste_scores = []
    amount_scores = []
    delivery_scores = []
    order_dates = []
    order_details = []
    customer_reviews = []
    owner_dates = []
    owner_reviews = []

    #  내부 리뷰 2~ 301번 리뷰까지 가져오기
    start = 2
    finish = 302
    
    # 외부(아래) 300개씩 eg) 620개 리뷰면, repeat_num_per_restaurant == 2번 + 20개 리뷰 
    # 4가 되어야 3번 반복
    for repeat_total in tqdm(range(1, int(repeat_num_per_restaurant), 1)):
        
         # 3번 중 '더보기' 30번을 2번 해야 한다.
        if repeat_total != (int(repeat_num_per_restaurant)-1):
            print("더보기 30번 누르기 " + str(repeat_num_per_restaurant-1) + "번 반복 중 " + str(repeat_total) + "번째 실행중")
            for more in tqdm(range(1,31,1)):
                try:
                    btn=driver.find_element_by_css_selector('span>i.arr-down')
                    btn.click()
                    time.sleep(2.5)
                except:
                    break
        # 3번 중 '더보기'  나머지 2번을 해야  20개 리뷰가 보인다.
        else:
            rest = int((finish%300)/10)
            print("더보기 나머지" + str(rest) +  "번 실행" )
            for more in tqdm(range(1,rest,1)):
                try:
                    btn=driver.find_element_by_css_selector('span>i.arr-down')
                    btn.click()
                    time.sleep(2.5)
                except:
                    break
            
            

        print(str(start-1) + "부터 " + str(finish-2) + "까지 크롤링중")
        
        for j in tqdm(range(start, finish, 1)):

            try:
                taste_score = driver.find_elements_by_css_selector(f'#review > li:nth-child({j}) > div:nth-child(2) > div > span.category > span:nth-child(3)')[0].text.strip()
            except:
                taste_score = 0
            try:
                amount_score = driver.find_elements_by_css_selector(f'#review > li:nth-child({j}) > div:nth-child(2) > div > span.category > span:nth-child(6)')[0].text.strip()
            except:
                amount_score = 0
            try:
                delivery_score = driver.find_elements_by_css_selector(f'#review > li:nth-child({j}) > div:nth-child(2) > div > span.category > span:nth-child(9)')[0].text.strip()
            except:
                delivery_score = 0
            try:
                order_date = driver.find_elements_by_css_selector(f'#review > li:nth-child({j}) > div:nth-child(1) > span.review-time.ng-binding')[0].text.strip()
            except:
                order_date = 0
            try:
                order_detail = driver.find_elements_by_css_selector(f'#review > li:nth-child({j}) > div.order-items.default.ng-binding')[0].text.strip()
            except:
                order_detail = 0
            try:
                customer_review = driver.find_elements_by_css_selector(f'#review > li:nth-child({j}) > p')[0].text.strip()
            except:
                customer_review = 0
            try:
                owner_date = driver.find_elements_by_css_selector(f'#review > li:nth-child({j}) > div.review-answer.ng-scope > div > span.review-time.ng-binding')[0].text.strip()
            except:
                owner_date = 0    
            try:
                owner_review = driver.find_elements_by_css_selector(f'#review > li:nth-child({j}) > div.review-answer.ng-scope > p')[0].text.strip()
            except:
                owner_review = 0

            taste_scores.append(taste_score)
            amount_scores.append(amount_score)
            delivery_scores.append(delivery_score)
            order_dates.append(order_date)
            order_details.append(order_detail)
            customer_reviews.append(customer_review)
            owner_dates.append(owner_date)
            owner_reviews.append(owner_review)
            time.sleep(0.2)

        start = finish
        
        # 아.. 이렇게 하면 안되는데 방법이 없다.
        
        if repeat_total > (int(repeat_num_per_restaurant)-1):
            finish = int(finish) + 300
        else:
            finish = int(finish) + int(int(customer_num)%300)

        time.sleep(2)



    time.sleep(2) 
    restaurant_info = {'taste':taste_scores,'amount':amount_scores,
                    'delivery':delivery_scores,'order':order_details,
                    'order_date':order_dates,'customer':customer_reviews,
                    'owner_date':owner_dates,'owner':owner_reviews}
    restaurant_info_toCSV = pd.DataFrame(restaurant_info)
    restaurant_info_toCSV.to_csv(f"{restaurant_name}.csv", encoding="utf-16")
    
    driver.back()
    time.sleep(2)


59번 식당부터, 61번 식당까지, 2개의 식당 중, 59번 식당 크롤링 중
얼음냉면&간편컵밥 388개의 리뷰 크롤링


더보기 30번 누르기 2번 반복 중 1번째 실행중



1부터 300까지 크롤링중



더보기 나머지10번 실행


301부터 388까지 크롤링중




59번 식당부터, 61번 식당까지, 2개의 식당 중, 60번 식당 크롤링 중
라곱창 337개의 리뷰 크롤링


더보기 30번 누르기 2번 반복 중 1번째 실행중


In [14]:
int(int(customer_num)%300)

6

In [17]:
int(repeat_num_per_restaurant)-1

2

In [18]:
repeat_total

2

In [19]:
int(int(customer_num)%300)


213

In [20]:
finish

815